In [24]:
import pandas as pd 
import numpy as np
import re

In [25]:
data = """
var `transmissionZwZm' = 2500
var `transmissionZmMz' = 0
var `transmissionMzZm' = 0
var `transmissionZwMz' = 2500
var `transmissionMzZw' = 0
var `transmissionZmZw' = 0
var `WindZmcapacity' = 2000
var `OilMzcapacity' = 5000
var `BiomassZwcapacity' = 8000
var `BiomassZmcapacity' = 1000
var `WindZwcapacity' = 2000
var `Hydro_RunOfRiverZmcapacity' = 3000
var `BiomassMzcapacity' = 1000
var `Solar_PVZwcapacity' = 5000
var `CCGTZmcapacity' = 2000
var `Hydro_RunOfRiverZwcapacity' = 2000
var `OilZmcapacity' = 4000
var `Hydro_RunOfRiverMzcapacity' = 4000
var `OilZwcapacity' = 2000
var `HardCoalZmcapacity' = 1000
var `HardCoalZwcapacity' = 1000
var `Solar_PVZmcapacity' = 5000
var `CCGTZwcapacity' = 1000
var `Solar_PVMzcapacity' = 5000
var `CCGTZmcapFactor' = 60
var `BiomassZwcapFactor' = 60
var `WindZmcapFactor' = 30
var `HardCoalZwcapFactor' = 70
var `HardCoalMzcapacity' = 1000
var `Hydro_RunOfRiverZmcapFactor' = 40
var `CCGTZwcapFactor' = 60
var `HardCoalZmcapFactor' = 60
var `CCGTMzcapacity' = 2000
var `Hydro_RunOfRiverZwcapFactor' = 40
var `OilZmcapFactor' = 40
var `OilZwcapFactor' = 40
var `WindMzcapacity' = 3000
var `BiomassZmcapFactor' = 60
var `Solar_PVZwcapFactor' = 20
var `Hydro_RunOfRiverMzcapFactor' = 40
var `Solar_PVZmcapFactor' = 20
var `WindZwcapFactor' = 30
var `OilMzcapFactor' = 40
var `Solar_PVMzcapFactor' = 20
var `BiomassMzcapFactor' = 60
var `WindMzcapFactor' = 30
var `HardCoalMzcapFactor' = 70
var `CCGTMzcapFactor' = 60
"""


In [26]:
# Dictionary to store capacity and capacity factor values
capacity_data = {}
cap_factor_data = {}
transmission_data = []

# Extract all values
matches = re.findall(r"var `(\w+)' = (\d+)", data)

for var, value in matches:
    value = int(value)

    # Check if it's a transmission value
    if var.startswith("transmission"):
        # Extract the two-country names (e.g., transmissionZwMz → From: Zw, To: Mz)
        match = re.match(r"transmission(\w+)(\w+)", var)
        if match:
            from_country, to_country = match.groups()
            transmission_data.append([from_country, to_country, value])

    # Check if it's a capacity value
    elif "capacity" in var:
        capacity_data[var] = value

    # Check if it's a capacity factor value
    elif "capFactor" in var:
        cap_factor_data[var] = value

# List to store power plant data
power_plant_data = []

# Process capacity data and match with capFactor
for var, cap in capacity_data.items():
    # Extract country and power plant type
    match = re.match(r"(\w+)(Zm|Zw|Mz)capacity", var)
    if match:
        power_type, country = match.groups()
        base_name = var.replace("capacity", "capFactor")  # Find matching capFactor
        cap_factor_value = cap_factor_data.get(base_name, 0)  # Default to 0 if missing

        power_plant_data.append([country, power_type, cap_factor_value, cap])

# Create DataFrames
df_power_plants = pd.DataFrame(power_plant_data, columns=["Country", "PowerPlantType", "CapFactor", "Capacity"])
df_transmissions = pd.DataFrame(transmission_data, columns=["From", "To", "Capacity"])

# Replace country codes with full names in transmission data
df_transmissions['To'] = df_transmissions.apply(lambda row: row['From'][-1] + row['To'], axis=1)
df_transmissions['From'] = df_transmissions['From'].apply(lambda x: x[:-1])

In [27]:
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.graph_objects as go

# Get unique countries
unique_countries = df_power_plants['Country'].unique()
num_countries = len(unique_countries)

# Create subplots with "domain" type for pie charts
fig = make_subplots(
    rows=1, 
    cols=num_countries, 
    subplot_titles=unique_countries,
    specs=[[{"type": "domain"}] * num_countries]  # "domain" type is required for pie charts
)

# Create a pie chart for each country and add to subplots
for i, country in enumerate(unique_countries):
    df_country = df_power_plants[df_power_plants['Country'] == country]
    pie = px.pie(
        df_country, 
        values='Capacity', 
        names='PowerPlantType', 
        title=f'Power Plant Capacity in {country}', 
        template='plotly_dark'  # Ensures dark theme
    )

    # Add pie chart traces to subplot
    for trace in pie.data:
        fig.add_trace(trace, row=1, col=i+1)

# Update layout with dark theme
fig.update_layout(
    height=600, 
    width=1200, 
    title_text="Power Plant Capacity Distribution by Country",
    template='plotly_dark'  # Ensures the entire figure follows the dark theme
)

fig.show()

In [28]:
# Calculate total capacity per country
df_power_plants['AllocatedCapacity'] = df_power_plants['Capacity'] * df_power_plants['CapFactor'] / 100
total_capacity_per_country = df_power_plants.groupby('Country')['AllocatedCapacity'].sum().reset_index().rename(columns={'AllocatedCapacity': 'Capacity'})

# Plot the bar graph
fig_total_capacity = px.bar(
    total_capacity_per_country,
    x='Country',
    y='Capacity',
    title='Total Capacity Allocated per Country',
    template='plotly_dark'  # Ensures dark theme
)

fig_total_capacity.update_layout(
    xaxis_title='Country',
    yaxis_title='Total Capacity'
)

fig_total_capacity.show()

In [29]:
# Calculate total transmission capacity sent and received per country
df_transmission_sent = df_transmissions.groupby('From')['Capacity'].sum().reset_index().rename(columns={'From': 'Country', 'Capacity': 'SentCapacity'})
df_transmission_received = df_transmissions.groupby('To')['Capacity'].sum().reset_index().rename(columns={'To': 'Country', 'Capacity': 'ReceivedCapacity'})

# Merge the dataframes
df_transmission_capacity = pd.merge(df_transmission_sent, df_transmission_received, on='Country', how='outer').fillna(0)
df_transmission_capacity = pd.merge(df_transmission_capacity, total_capacity_per_country, on='Country', how='outer').fillna(0)

# Calculate net capacity
df_transmission_capacity['NetCapacity'] = df_transmission_capacity['Capacity'] + df_transmission_capacity['ReceivedCapacity'] - df_transmission_capacity['SentCapacity']

# Plot the bar graph
fig_transmission_capacity = px.bar(
    df_transmission_capacity,
    x='Country',
    y=['SentCapacity', 'ReceivedCapacity', 'NetCapacity'],
    title='Total Transmission Capacity Sent, Received, and Net per Country',
    template='plotly_dark'  # Ensures dark theme
)

# Add horizontal lines for each country
fig_transmission_capacity.add_shape(
    type="line",
    x0=-0.5, x1=0.5, y0=10407, y1=10407,
    line=dict(color="RoyalBlue", width=2),
    xref="x", yref="y"
)

fig_transmission_capacity.add_shape(
    type="line",
    x0=0.5, x1=1.5, y0=9263, y1=9263,
    line=dict(color="RoyalBlue", width=2),
    xref="x", yref="y"
)

fig_transmission_capacity.add_shape(
    type="line",
    x0=1.5, x1=2.5, y0=4291, y1=4291,
    line=dict(color="RoyalBlue", width=2),
    xref="x", yref="y"
)

fig_transmission_capacity.update_layout(
    xaxis_title='Country',
    yaxis_title='Total Transmission Capacity',
    barmode='group'  # Group bars together
)

fig_transmission_capacity.show()